In [43]:
df = read.csv("dat.csv")
df["previous_qualification"] = as.factor(df$previous_qualification)
df["mother_s_qualification"] = as.factor(df$mother_s_qualification)
df["father_s_qualification"] = as.factor(df$father_s_qualification)
df["mother_s_occupation"] = as.factor(df$mother_s_occupation)
df["father_s_occupation"] = as.factor(df$father_s_occupation)
df["educational_special_needs"] = as.factor(df$educational_special_needs)
df["gender"] = as.factor(df$gender)
df["international"] = as.factor(df$international)
df["target"] = as.factor(df$target)
head(df)

,X,previous_qualification,previous_qualification_grade_,mother_s_qualification,father_s_qualification,mother_s_occupation,father_s_occupation,admission_grade,educational_special_needs,gender,age_at_enrollment,international,unemployment_rate,inflation_rate,gdp,target
,<int>,<fct>,<dbl>,<fct>,<fct>,<fct>,<fct>,<dbl>,<fct>,<fct>,<int>,<fct>,<dbl>,<dbl>,<dbl>,<fct>
1,1,1,122.0,19,12,5,9,127.3,0,1,20,0,10.8,1.4,1.74,Dropout
2,2,1,160.0,1,3,3,3,142.5,0,1,19,0,13.9,-0.3,0.79,Graduate
3,3,1,122.0,37,37,9,9,124.8,0,1,19,0,10.8,1.4,1.74,Dropout
4,4,1,122.0,38,37,5,3,119.6,0,0,20,0,9.4,-0.8,-3.12,Graduate
5,5,1,100.0,37,38,9,9,141.5,0,0,45,0,13.9,-0.3,0.79,Graduate
6,6,19,133.1,37,37,9,7,114.8,0,1,50,0,16.2,0.3,-0.92,Graduate


In [29]:
install.packages("twang")
install.packages("cobalt")
install.packages("twangContinuous")
install.packages("svyGAM")
install.packages("survey")

Installing package into ‘/Users/chittaranjan/Library/R/arm64/4.1/library’
(as ‘lib’ is unspecified)




The downloaded binary packages are in
	/var/folders/0b/727m7dn55nxdtphddkmjzdbr0000gn/T//RtmpUO3s1u/downloaded_packages


Installing package into ‘/Users/chittaranjan/Library/R/arm64/4.1/library’
(as ‘lib’ is unspecified)




The downloaded binary packages are in
	/var/folders/0b/727m7dn55nxdtphddkmjzdbr0000gn/T//RtmpUO3s1u/downloaded_packages


Installing package into ‘/Users/chittaranjan/Library/R/arm64/4.1/library’
(as ‘lib’ is unspecified)




The downloaded binary packages are in
	/var/folders/0b/727m7dn55nxdtphddkmjzdbr0000gn/T//RtmpUO3s1u/downloaded_packages


Installing package into ‘/Users/chittaranjan/Library/R/arm64/4.1/library’
(as ‘lib’ is unspecified)




The downloaded binary packages are in
	/var/folders/0b/727m7dn55nxdtphddkmjzdbr0000gn/T//RtmpUO3s1u/downloaded_packages


In [16]:
library(twangContinuous)
library(cobalt)
library(survey)

In [18]:
colnames(df)

[1] "X"                             "previous_qualification"       
 [3] "previous_qualification_grade_" "mother_s_qualification"       
 [5] "father_s_qualification"        "mother_s_occupation"          
 [7] "father_s_occupation"           "admission_grade"              
 [9] "educational_special_needs"     "gender"                       
[11] "age_at_enrollment"             "international"                
[13] "unemployment_rate"             "inflation_rate"               
[15] "gdp"                           "target"

ps.cont is a way of getting propensity scores for a continuous treatment variables (admission_grade in our case). Propensity scores are a probability of how likely a particular observation is to have that value of the treatment (check this definition)

In [19]:
psc.out <- ps.cont(admission_grade ~ previous_qualification +
                   previous_qualification_grade_ + 
                   mother_s_qualification + 
                   father_s_qualification +
                   mother_s_occupation +
                   father_s_occupation +
                   educational_special_needs +
                   gender +
                   age_at_enrollment +
                   international +
                   unemployment_rate +
                   inflation_rate +
                   gdp
                   , data = df)
summary(psc.out)

,n,ess,max.wcor,mean.wcor,rms.wcor,iter
unw,4424,4424.000,0.5804442,0.02521831,0.05633220,NA
AAC,4424,4068.986,0.3532101,0.02371980,0.04287533,33


In [21]:
head(df$admission_grade)

[1] 127.3 142.5 124.8 119.6 141.5 114.8

`w` are the weights, and there is one for each observation of the data. We use this to perform the survey weighted glm.

In [35]:
head(psc.out$w)

1         2         3         4         5         6 
0.9229765 0.6922250 0.9048052 0.8846176 1.3263157 0.9230411

In [45]:
library(svyVGAM)
design <- svydesign(ids=~1, weights=psc.out$w, data=df)
mmodel <- svy_vglm(target ~ admission_grade, family=multinomial, design=design)

In [46]:
summary(mmodel)

svy_vglm.survey.design(target ~ admission_grade, family = multinomial, 
    design = design)
Independent Sampling design (with replacement)
svydesign(ids = ~1, weights = psc.out$w, data = df)
                        Coef         SE       z         p
(Intercept):1      1.8580940  0.3609064  5.1484 2.627e-07
(Intercept):2      1.0059447  0.3986204  2.5236   0.01162
admission_grade:1 -0.0178421  0.0028650 -6.2276 4.735e-10
admission_grade:2 -0.0158937  0.0031429 -5.0570 4.258e-07

Interpretation: With 1 point increase in admission_grade, chance of dropout (`admission_grade:1`) goes down by 1.7% and chance of still being enrolled (failed to graduate in stipulated time) goes down by 1.5%